## Lab 04

## Zadanie 1 (Igor Sieradzki)

Funkcja <t>flow_rate</i> do modyfikacji

In [220]:
def flow_rate(weight, time, *, period=1, units_per_kg=1):
    """ Funkcja wylicza ile wagi produktu przybyło/ubyło w jednostce czasu """
    return weight * units_per_kg / (time / period) 

weight = 0.5
time = 3

In [221]:
flow = flow_rate(weight, time)
print("{0:.3} kg per second".format(flow))

0.167 kg per second


In [222]:
Zmodyfikuj funckje flow_rate tak, aby poniższe wywołania działały poprawnie

SyntaxError: invalid syntax (<ipython-input-222-fbf05e22c559>, line 1)

In [223]:
flow = flow_rate(weight, time, period=60, units_per_kg=1000)
print("{} grams per minute".format(flow))

10000.0 grams per minute


In [224]:
flow = flow_rate(weight, time, period=1, units_per_kg=1)
print("{0:.3} kg per second".format(flow))

0.167 kg per second


In [225]:
flow = flow_rate(weight, time)
print("{0:.3} grams per minute".format(flow))

0.167 grams per minute


Wyjaśnić czemu poniższe rzucanie błędu jest porządane

In [59]:
try:
    flow = flow_rate(weight, time, 3600, 2.2)
except TypeError:
    print(True)

True


## Zadanie 2 (Igor Sieradzki)

Dopisz definicje dekoratora _timeit_, który wypisze na wyjscie standardowe czas wywołania udekorowanej funckji np.

```
@timeit
def foo(x):
    return x**2
    
r = foo(2)
```
wypize: <br>
    `Function foo took: 0.00001 seconds` <br>
oraz zwróci 4

In [226]:
from time import time

def timeit(func):
    """ Wypisuje czas wywołania udekorowanej funckji """
    def inner(*args, **kwargs):
        t1 = time()
        res = func(*args, **kwargs)
        t2 = time()
        print(f'Function {func.__name__} took: {t2 - t1:.5} seconds')
        return res
    return inner

@timeit
def squares_list(n):
    squares = []
    for i in range(n):
        squares.append(i ** 2)
    return squares

@timeit
def squares_comprehension(n):
    return [i ** 2 for i in range(n)]

@timeit
def squares_map(n):
    return map(lambda x: x**2, range(n))

n = 1000000
l = squares_list(n)
c = squares_comprehension(n)
m = squares_map(n)


Function squares_list took: 0.40796 seconds
Function squares_comprehension took: 0.35592 seconds
Function squares_map took: 3.0994e-06 seconds


## Zadanie 3 (Igor Sieradzki)

Dopisz definicje dekoratora _derivate_ wg. instrukcji w _docstringu_

In [229]:
import sys
sys.float_info.epsilon # epsilon maszynowy


def derivate(epsilon=None):
    """
    Zwraca pochodną funkcji w punkcie, wg. wzoru f'(x) = [f(x+h) - f(x)]/h, 
    gdzie h jest parametrem dekoratora, jeśli nie zostanie podany, należy przyjąć 1000 * epsilon maszynowy
    """
    def wrapper(func): 
        def inner(arg):
            h = epsilon if epsilon else 1000 * sys.float_info.epsilon
            return (func(arg + h) - func(arg)) / h
        return inner
    return wrapper


@derivate(0.01)
def f(x):
    return x*x

@derivate(0.00001)
def g(x):
    return x*x*x+3

def test(a, b, eps=1):
    return abs(round(a)-round(b)) < eps

print(test(f(100), 200.0))
print(round(f(0)) == 0.0)

print(test(g(100), 30000.0))
print(round(g(0)) == 0.0)

from random import random
for x in [random()*1000. for _ in range(20)]:
    print(f(x), 2*x, '\t', test(f(x), 2*x))
    print(g(x), 3*x**2, '\t', test(g(x), 3*x**2))

True
True
True
True
775.129189772997 775.119189775683 	 True
450607.32886195177 450607.31876888353 	 True
1402.6151010708418 1402.6051010709202 	 True
1475475.8238792417 1475475.802162625 	 True
1792.207194736693 1792.1971947323955 	 True
2408978.1165122986 2408978.0886050006 	 True
471.69988501555054 471.6898850165907 	 True
166868.51773411036 166868.51072022345 	 True
1873.2046677498147 1873.1946677525239 	 True
2631643.7244415283 2631643.697472366 	 True
201.1697348942107 201.15973489420645 	 True
30348.93223084509 30348.929207030567 	 True
887.2705961024622 887.2605961030287 	 True
590423.5363006592 590423.5240478263 	 True
1312.3986672377214 1312.388667241687 	 True
1291773.0271816254 1291773.0104283087 	 True
1335.9346752229612 1335.9246752190684 	 True
1338521.0752487183 1338521.05339438 	 True
190.8382467498086 190.82824674977573 	 True
27311.56768277287 27311.56481819497 	 True
1128.1620381458197 1128.152038146544 	 True
954545.2833175658 954545.2658806511 	 True
111.635114731

## Zadanie 4 (Igor Sieradzki)

Dopisz definicje dekoratora _accepts_ wg. instrukcji w _docstringu_

In [230]:
def accepts(*types):
    """Sprawdza czy udekorowanej funckji zostały podane odpowiednie parametry zdefiniowane 
       w argumentach dekoratora"""
    def wrapper(func):
        def inner(*args, **kwargs):
            all_args = list(args) + [v for v in kwargs.values()]
            if len(types) == len(all_args):
                if all(isinstance(a, t) for a, t in zip(all_args, types)):
                    return func(*args, **kwargs)
            raise TypeError()
        return inner
    return wrapper


@accepts(str)
def capitalize(word):
    return word[0].upper() + word[1:]

print(capitalize('ola') == 'Ola')

try:
    capitalize(2)
except TypeError:
    print(True)

@accepts(float, int)
def static_pow(base, exp):
    return base ** exp 

print(static_pow(2., 2) == 4.)
print(static_pow(2., exp=2) == 4.)
print(static_pow(base=2., exp=2) == 4.)

try:
    static_pow('x', 10)
except TypeError:
    print(True)
    
try:
    static_pow(2, 2.2)
except TypeError:
    print(True)

True
True
True
True
True
True
True


## Zadanie 5 (Igor Sieradzki)

Dopisz definicje dekoratora _returns_ wg. instrukcji w _docstringu_

In [239]:
def returns(*types):
    """Sprawdza czy udekorowana funkcja zwraca poprawne argumenty, zdefiniowane w parametrach dekoratora"""
    def wrapper(func):
        def inner(*args, **kwargs):
            res = func(*args, **kwargs)
            if (
                not isinstance(res, tuple) and len((res,)) != len(types)
                or isinstance(res, tuple) and len(res) != len(types)
            ):
                raise TypeError()

            if all(isinstance(a, t) for a, t in zip(res, types)):
                return res
            else:
                raise TypeError()
        return inner
    return wrapper


@returns(int)
def str_only_identity(word):
    return 1

print(str_only_identity('hello') == 'hello')

try:
    str_only_identity(10)
except TypeError:
    print(True)
    
@returns(int, int)
def split_indices(x):
    return x[0], x[1]

print(split_indices(x=[6,9]) == (6,9))

try:
    split_indices('AB')
except TypeError:
    print(True)

TypeError: 'int' object is not iterable

## Zadanie 6 (Igor Sieradzki)
Stwórz dekorator cached służący do cachowania wywołań dowolnej funkcji, tzn. chcemy by:
* wywołanie funkcji z określonymi argumentami miało miejsce tylko raz
* funkcja mogła przyjmować dowolną liczbę nazwanych i nienazwanych argumentów
* nie musi reagować poprawnie na domyślne argumenty, tzn. wywołanie funkcji z domyślnymi argumentami a podanie dokładnie takich samych może być traktowane jako dwa różne wywołania
* na opakowanej funkcji można wywołać `.cache_reset()`, który usunie cache z pamięci
* wywołanie `.cache_status()` zwraca string z opisem w postaci: <br>
    `Function FUNCTION_NAME called X times, evaluated Y times`

In [ ]:
from random import random

@cached
def foo(x, y=1, z=4):
    return random()
        
print(foo(3) == foo(3))
print(foo(4) == foo(4))
print(foo(3, z=-1, y=3) == foo(3, y=3, z=-1))
print(foo(3) != foo(x=3))
a = foo(3)
foo.cache_reset()
print(a != foo(3))
print(foo.cache_status() == 'Function foo called 10 times, evaluated 5 times')

## Zadanie 7 (Krzysztof Hajto)

Napisz dekorator który będzie robić n-krotne złożenie funkcji, gdzie n jest parametrem dekoratora

In [ ]:
def zlozenie(n):
    pass

@zlozenie(3)
def f1(x):
    return x+1

@zlozenie(2)
def f2(x):
    return x*x

@zlozenie(5)
def f3(word):
    return "".join(chr(ord(l)+1) for l in word)

print(f1(2)==5)
print(f2(3)==81)
print(f3("alamakota")=="fqfrfptyf")

## Zadanie 8 (Krzysztof Hajto)

Python nie ma wbudowanej instrukcji switch. Ale posiada anonimowe funkcje oraz słowniki. Zaimplementuj poniższy switch w postaci słownika funkcji.
`
int my_function(x, y) {


    switch(x) {
        case 1: return y*y;
        case 2: return x+y;
        case 3: return x*y;
        case 4: return 0;
    }
}
`

PS. Nigdy nie róbcie tego w faktycznym kodzie :)

In [ ]:
def my_function(x, y):
    pass

print(my_function(1,3)==9)
print(my_function(2,4)==6)
print(my_function(3,1)==3)
print(my_function(4,9)==0)